In [7]:
import cv2
img_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\test\268.jpeg'
img = cv2.imread(img_path)
img_height, img_width = img.shape[:2]

In [8]:
img_height, img_width

(1351, 1013)

In [9]:
from ultralytics import YOLO

model = YOLO(r"model\train_yolo11s_roi\weights\best.pt") 
results = model.predict(source=img, iou = 0.1)


0: 640x480 1 plate, 142.7ms
Speed: 5.4ms preprocess, 142.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


In [10]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen


In [11]:
result.boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.8680])
data: tensor([[418.6080, 566.6142, 715.3104, 654.9916,   0.8680,   0.0000]])
id: None
is_track: False
orig_shape: (1351, 1013)
shape: torch.Size([1, 6])
xywh: tensor([[566.9592, 610.8029, 296.7024,  88.3774]])
xywhn: tensor([[0.5597, 0.4521, 0.2929, 0.0654]])
xyxy: tensor([[418.6080, 566.6142, 715.3104, 654.9916]])
xyxyn: tensor([[0.4132, 0.4194, 0.7061, 0.4848]])

In [15]:
import os
# --- Paths ---
#label_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\sampled_OCR Meter Reading.v1i.yolov12\train\labels'
#img_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\sampled_OCR Meter Reading.v1i.yolov12\train\images'
output_crop_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped'
#output_label_path = r'C:\Your\Output\Crop\Labels'
os.makedirs(output_crop_path, exist_ok=True)
#os.makedirs(output_label_path, exist_ok=True)

In [16]:
b_xywhn = result.boxes.xywhn[0] # มันอาจมีหลาย boxes; [0] คือ box แรก
b_conf = result.boxes.conf[0]
b_cls = result.boxes.cls[0]

In [17]:
h_img, w_img = img.shape[:2]

xc, yc, w, h = (b_xywhn[i] for i in range(4))
xc, yc, w, h = xc*w_img, yc*h_img, w*w_img, h*h_img
xc, yc, w, h

(tensor(566.9592), tensor(610.8029), tensor(296.7024), tensor(88.3774))

In [20]:
margin_w = 0.05 * w  # 5% of width
margin_h = 0.05 * h  # 5% of height
x1, y1 = max(0, int(xc-w/2-margin_w)), max(0, int(yc-h/2-margin_h))
x2, y2 = min(int(xc+w/2+1+margin_w), w_img), min(int(yc+h/2+1+margin_h), h_img)

cropped = img[y1:y2, x1:x2]

cv2.imwrite(output_crop_path, cropped)  # เซฟไว้ดู

False

In [23]:
i = 0

In [24]:
import time
i += 1
filename = f"crop_{i}.jpg"
full_path = os.path.join(output_crop_path, filename)
cv2.imwrite(full_path, cropped)


True

In [25]:
def pred_roi(model_path, img, iou = 0.2):
    model = YOLO(model_path) 
    return model.predict(source=img, iou = iou)

In [26]:
import cv2
from ultralytics import YOLO

img = cv2.imread(r'test\229.jpeg')
h_img, w_img = img.shape[:2]

model = YOLO(r"model\train_yolo11s_roi\weights\best.pt") 
results = model.predict(source=img, iou = 0.1)

cropped_imgs = []

for result in results: #แจกแจงทีละรูป
    for box in result.boxes.xywhn: #แจกแจงทีละ box
        xc, yc, w, h = box
        margin_w = 0.05 * w  # 5% of width
        margin_h = 0.05 * h  # 5% of height
        x1 = int(xc - w / 2 -margin_w); x1 = max(0,x1)
        y1 = int(yc - h / 2 -margin_h); y1 = max(0, y1)
        x2 = int(xc + w / 2 +margin_w); x2 = min(w_img, x2)
        y2 = int(yc + h / 2 +margin_h); y2 = min(h_img, y2)
        cropped = img[y1:y2, x1:x2]
        cropped_imgs.append(cropped)


0: 640x480 1 plate, 137.4ms
Speed: 8.8ms preprocess, 137.4ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 480)


In [27]:
import os
label_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\sampled_OCR Meter Reading.v1i.yolov12\train\labels'
label_list = [f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))]

img_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\sampled_OCR Meter Reading.v1i.yolov12\train\images'
img_list = [f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]

In [3]:
label_list

['-1024x768_jpg.rf.32ecbc38708881076e24abe9e72f87f8.txt',
 '02_jpg.rf.f374c18616db201cc5e0c38637000f51.txt',
 '03_jpg.rf.99f438df7c6e18bf3ca46b2cb4214d23.txt',
 '04676_png_jpg.rf.b8520ab1f613abddc2c55b78e1f3c559.txt',
 '04939_png_jpg.rf.6d09964dfc4952c7f70131599a200e7c.txt',
 '0658724_png_jpg.rf.bf0dbda221acb15817322282cb9751c4.txt',
 '07374_png_jpg.rf.3818072de546c76ad004f7a309ccc409.txt',
 '07757_png_jpg.rf.b0a21d2284400d46334677dd0bdb06e2.txt',
 '08_jpg.rf.f90d2f15661cef329077f7cbf60ee52e.txt',
 '11_jpg.rf.3fcc1d5cde467690fc295c819fcb06e3.txt',
 '1417360467-image-o_jpg.rf.19a97675c0b30f9334996913fd6242f4.txt',
 '14499_png_jpg.rf.e4ed8572291d7a5738876f92f36cc70e.txt',
 '1483053_png_jpg.rf.2f8cc31ea4bf4d381882ef15c927299e.txt',
 '17_jpg.rf.7747a5855a2a5fa850b562cfeff304c8.txt',
 '18261_png_jpg.rf.7e44b1d75df5754b4bab5f6a321db95f.txt',
 '22986_png_jpg.rf.c4c8d2c7198f788aa6ae4fe18bb3ff46.txt',
 '234132195_865874044306445_494124124440194450_n_jpg.rf.0cc0f093d24f33913bc50e859c815cf3.txt',

In [4]:
img_list

['-1024x768_jpg.rf.32ecbc38708881076e24abe9e72f87f8.jpg',
 '02_jpg.rf.f374c18616db201cc5e0c38637000f51.jpg',
 '03_jpg.rf.99f438df7c6e18bf3ca46b2cb4214d23.jpg',
 '04676_png_jpg.rf.b8520ab1f613abddc2c55b78e1f3c559.jpg',
 '04939_png_jpg.rf.6d09964dfc4952c7f70131599a200e7c.jpg',
 '0658724_png_jpg.rf.bf0dbda221acb15817322282cb9751c4.jpg',
 '07374_png_jpg.rf.3818072de546c76ad004f7a309ccc409.jpg',
 '07757_png_jpg.rf.b0a21d2284400d46334677dd0bdb06e2.jpg',
 '08_jpg.rf.f90d2f15661cef329077f7cbf60ee52e.jpg',
 '11_jpg.rf.3fcc1d5cde467690fc295c819fcb06e3.jpg',
 '1417360467-image-o_jpg.rf.19a97675c0b30f9334996913fd6242f4.jpg',
 '14499_png_jpg.rf.e4ed8572291d7a5738876f92f36cc70e.jpg',
 '1483053_png_jpg.rf.2f8cc31ea4bf4d381882ef15c927299e.jpg',
 '17_jpg.rf.7747a5855a2a5fa850b562cfeff304c8.jpg',
 '18261_png_jpg.rf.7e44b1d75df5754b4bab5f6a321db95f.jpg',
 '22986_png_jpg.rf.c4c8d2c7198f788aa6ae4fe18bb3ff46.jpg',
 '234132195_865874044306445_494124124440194450_n_jpg.rf.0cc0f093d24f33913bc50e859c815cf3.jpg',

In [6]:
['.'.join(l.split('.')[:-1])+ '.jpg' for l in label_list]

['-1024x768_jpg.rf.32ecbc38708881076e24abe9e72f87f8.jpg',
 '02_jpg.rf.f374c18616db201cc5e0c38637000f51.jpg',
 '03_jpg.rf.99f438df7c6e18bf3ca46b2cb4214d23.jpg',
 '04676_png_jpg.rf.b8520ab1f613abddc2c55b78e1f3c559.jpg',
 '04939_png_jpg.rf.6d09964dfc4952c7f70131599a200e7c.jpg',
 '0658724_png_jpg.rf.bf0dbda221acb15817322282cb9751c4.jpg',
 '07374_png_jpg.rf.3818072de546c76ad004f7a309ccc409.jpg',
 '07757_png_jpg.rf.b0a21d2284400d46334677dd0bdb06e2.jpg',
 '08_jpg.rf.f90d2f15661cef329077f7cbf60ee52e.jpg',
 '11_jpg.rf.3fcc1d5cde467690fc295c819fcb06e3.jpg',
 '1417360467-image-o_jpg.rf.19a97675c0b30f9334996913fd6242f4.jpg',
 '14499_png_jpg.rf.e4ed8572291d7a5738876f92f36cc70e.jpg',
 '1483053_png_jpg.rf.2f8cc31ea4bf4d381882ef15c927299e.jpg',
 '17_jpg.rf.7747a5855a2a5fa850b562cfeff304c8.jpg',
 '18261_png_jpg.rf.7e44b1d75df5754b4bab5f6a321db95f.jpg',
 '22986_png_jpg.rf.c4c8d2c7198f788aa6ae4fe18bb3ff46.jpg',
 '234132195_865874044306445_494124124440194450_n_jpg.rf.0cc0f093d24f33913bc50e859c815cf3.jpg',

In [ ]:
import os

# กำหนด path ของโฟลเดอร์
ref_path = r'C:\Users\Windows 10\OneDrive - Toyota Asia\coding\202506 OCR\labels'
folder_path = r'C:\Users\Windows 10\OneDrive - Toyota Asia\coding\202506 OCR\data_fillshape'
prefix = 'book'

ref_list = [f for f in os.listdir(ref_path) if os.path.isfile(os.path.join(ref_path, f))]
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
ref_list.sort()
file_list.sort()  # เรียงตามชื่อไฟล์ก่อน เพื่อให้ได้ลำดับแน่นอน

In [2]:
import os
import cv2
import csv
import time
from ultralytics import YOLO
from datetime import datetime

# Define paths
RAW_IMAGE_DIR = r"C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\test"
CROPPED_IMAGE_DIR = r"C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR/project/cropped_images"
RESULT_CSV_PATH = r"C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR/project/results.csv"

# Create output directories if they don't exist
os.makedirs(CROPPED_IMAGE_DIR, exist_ok=True)

# Load YOLO models
model_roi = YOLO(r"model/train_yolo11s_roi/weights/best.pt")
model_number = YOLO(r"model/train_yolo11s_2/weights/best.pt")

# Supported image extensions
SUPPORTED_EXTENSIONS = (".jpg", ".jpeg", ".png")

# Function to crop ROI from image using model_roi
def crop_rois(image_path, image_array, image_name):
    results = model_roi.predict(source=image_array, iou=0.2)
    h_img, w_img = image_array.shape[:2]
    cropped_images = []

    for result_idx, result in enumerate(results):
        for box_idx, box in enumerate(result.boxes.xywhn):
            xc, yc, w, h = box
            margin_w = 0.05 * w
            margin_h = 0.05 * h
            x1 = int(xc * w_img - w * w_img / 2 - margin_w * w_img)
            y1 = int(yc * h_img - h * h_img / 2 - margin_h * h_img)
            x2 = int(xc * w_img + w * w_img / 2 + margin_w * w_img)
            y2 = int(yc * h_img + h * h_img / 2 + margin_h * h_img)

            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(w_img, x2)
            y2 = min(h_img, y2)

            cropped = image_array[y1:y2, x1:x2]
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            cropped_filename = f"{os.path.splitext(image_name)[0]}_roi{result_idx}_{box_idx}_{timestamp}.jpg"
            cropped_path = os.path.join(CROPPED_IMAGE_DIR, cropped_filename)
            success = cv2.imwrite(cropped_path, cropped)
            if success:
                cropped_images.append((cropped_path, cropped_filename, timestamp))
    return cropped_images

# Function to detect number from cropped image using model_number
def detect_number(image_path):
    results = model_number.predict(source=image_path, iou=0.2)
    for result in results:
        cwh = result.boxes.xywh
        num_order = cwh[:, 0].argsort()
        result_sorted = result.boxes.cls[num_order]
        result_sorted_str = ''.join([str(int(r.item())) for r in result_sorted])
    return result_sorted_str if result_sorted_str else "N/A"

# Main processing loop
with open(RESULT_CSV_PATH, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Cropped Filename", "Detected Number", "Timestamp"])

    for image_name in os.listdir(RAW_IMAGE_DIR):
        if image_name.lower().endswith(SUPPORTED_EXTENSIONS):
            image_path = os.path.join(RAW_IMAGE_DIR, image_name)
            image_array = cv2.imread(image_path)
            if image_array is None:
                continue

            cropped_images = crop_rois(image_path, image_array, image_name)
            for cropped_path, cropped_filename, timestamp in cropped_images:
                number = detect_number(cropped_path)
                writer.writerow([cropped_filename, number, timestamp])

print("Processing complete. Results saved to results.csv.")




0: 640x480 1 plate, 292.5ms
Speed: 35.9ms preprocess, 292.5ms inference, 20.2ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped_images\229_roi0_0_20250818_115013.jpg: 192x640 1 0, 1 1, 1 3, 2 4s, 1 6, 1 7, 148.9ms
Speed: 1.7ms preprocess, 148.9ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 640)

0: 640x480 1 plate, 147.0ms
Speed: 3.2ms preprocess, 147.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped_images\230_roi0_0_20250818_115013.jpg: 192x640 1 0, 1 1, 1 3, 1 4, 1 7, 2 8s, 113.8ms
Speed: 1.5ms preprocess, 113.8ms inference, 1.3ms postprocess per image at shape (1, 3, 192, 640)

0: 640x480 1 plate, 174.1ms
Speed: 3.6ms preprocess, 174.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped_

In [96]:
image_path = r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped_images\239_roi0_0_20250801_133416.jpg'
model_number = YOLO(r'C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\model\train_yolo11s_2\weights\best.pt')
results = model_number.predict(source=image_path, iou=0.2,
                               save=True, save_frames = True, save_txt = True, save_crop = True, save_conf = True,
                               project = 'inference', name = 'test_yolo11s_2_cropped_test')
for result in results:
    cwh = result.boxes.xywh
    num_order = cwh[:, 0].argsort()
    result_sorted = result.boxes.cls[num_order]
    result_sorted_str = ''.join([str(int(r.item())) for r in result_sorted])

result_sorted_str #if result_sorted_str else "N/A"


image 1/1 C:\Users\koraw\OneDrive - Toyota Asia\coding\202506 OCR\project\cropped_images\239_roi0_0_20250801_133416.jpg: 224x640 1 0, 2 1s, 1 3, 1 5, 2 9s, 68.9ms
Speed: 1.5ms preprocess, 68.9ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 640)
Results saved to inference\test_yolo11s_2_cropped_test
1 label saved to inference\test_yolo11s_2_cropped_test\labels


'0131959'

In [60]:
cwh = results[0].boxes.xywh; cwh

tensor([[129.0848,  51.7641,  28.2851,  40.4302],
        [167.3925,  52.0747,  30.9924,  42.4389],
        [236.3841,  57.1395,  31.3155,  55.4176],
        [ 90.0382,  49.8873,  24.6966,  38.0751],
        [ 51.6601,  51.1893,  23.7303,  39.3486],
        [200.9921,  53.2466,  24.9072,  44.4379],
        [275.1035,  45.3859,  24.3897,  27.5708]])

In [66]:
cwh[:, 0].argsort()

tensor([4, 3, 0, 1, 5, 2, 6])

In [79]:
result_sorted = results[0].boxes.cls[cwh[:, 0].argsort()]; result_sorted

tensor([0., 1., 3., 1., 9., 5., 9.])

In [91]:
result_sorted_str = ''.join([str(int(r.item())) for r in result_sorted]); result_sorted_str

'0131959'